In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
from os.path import join as pjoin
import os
import numpy as np
import pandas as pd

class microson_v1_dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, root_dir, split):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.split = split
        self.df = pd.read_csv(pjoin(root_dir, 'meta_microson_v1.csv'))
        self.df = self.df[self.df['mls_split'] == split]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        wavname = os.path.splitext(row.speech_path)[0]+'.wav'
        anechoic, _ = torchaudio.load(pjoin(pjoin(pjoin(self.root_dir, row.mls_split), 'anechoic'), wavname))
        reverberant, _ = torchaudio.load(pjoin(pjoin(pjoin(self.root_dir, row.mls_split), 'reverberant'), wavname))
        noise, _ = torchaudio.load(pjoin(pjoin(pjoin(self.root_dir, row.mls_split), 'noise'), wavname))

        return anechoic, reverberant, noise


In [40]:
hparams = {}
hparams['bs'] = 3

In [38]:
train_dataset = microson_v1_dataset('/home/ubuntu/Data/microson_v1', 'train')

In [41]:
train_loader = DataLoader(train_dataset, batch_size=hparams['bs'], shuffle=True)

In [42]:
for anechoics, reverberants, noises in train_loader:
    print(anechoics.shape)
    break

torch.Size([3, 2, 64000])
